In [2]:
import tkinter as tk
from tkinter import filedialog
import socket
import threading
from stegano import lsb
import cv2

def send_message():
    mode = mode_var.get()
    message = message_entry.get()

    # Open file dialog based on the selected mode
    if mode == "image":
        file_types = [("Image Files", "*.png *.jpg *.jpeg")]
    elif mode == "video":
        file_types = [("Video Files", "*.mp4 *.avi *.mov")]
    else:
        file_types = [("Audio Files", "*.mp3 *.wav *.ogg")]
        
    # Open file dialog based on the selected mode
    file_path = filedialog.askopenfilename(filetypes=file_types)
    
    if file_path:
        # Creating a socket connection
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_address = ('localhost', 8000)
        client_socket.connect(server_address)

        #send the mode to client
        client_socket.sendall(mode.encode())
        
        # Encoding a message in an image
        if mode == "image":         
            encoded_image = lsb.hide(file_path, message, auto_convert_rgb=True)
            encoded_image.save("encoded_img.png")
            with open("encoded_img.png", 'rb') as file:
                file_read = file.read()

        # Sending the message and mode of encoding to the other client
        client_socket.sendall(file_read)

        # Closing the socket connection
        client_socket.close()
    else:
        response_label.configure(text="No file selected.")
    return

def receive_thread():
    # Creating a socket connection
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 8001)
    server_socket.bind(server_address)
    server_socket.listen(1)
    
    while True:
        # Accepting a client connection
        client_socket, client_address = server_socket.accept()
        
        #recieve the mode of encoding
        mode = client_socket.recv(1024).decode()
        
        if mode == "image":
            # Receiving the message and mode of encoding from the other client
            file = open("recieved_img.png", "wb")
            data = client_socket.recv(1024)
            while data:
                file.write(data)
                data = client_socket.recv(1024)
            
            file.close()
            
            # Decoding the message from the encoded image
            encoded_image = "recieved_img.png"
            decoded_message = lsb.reveal(encoded_image)
            message = decoded_message
            
        elif mode == "video":
           pass
            
        # Displaying the received message and mode of encoding
        received_label.configure(text=f"Received message: {message}")
        mode_label.configure(text=f"Mode of encoding: {mode}")

    # Closing the socket connection
    client_socket.close()

# Creating the client UI
client = tk.Tk()
client.title("Client - 2")
client.geometry("400x200")  # Set the width to 800 and height to 600

# Message input
message_label = tk.Label(client, text="Enter your message:")
message_label.pack()
message_entry = tk.Entry(client)
message_entry.pack()


# Message input
message_label = tk.Label(client, text="Select a file:")
message_label.pack()

# Mode of encoding input
mode_label = tk.Label(client, text="Select the mode of encoding:")
mode_label.pack()

# Mode of encoding radio buttons
mode_var = tk.StringVar(value="text")
image_radio = tk.Radiobutton(client, text="Image", variable=mode_var, value="image")
image_radio.pack()  
video_radio = tk.Radiobutton(client, text="Video", variable=mode_var, value="video")
video_radio.pack()
text_radio = tk.Radiobutton(client, text="Audio", variable=mode_var, value="audio")
text_radio.pack()

# Send button
send_button = tk.Button(client, text="Send", command=send_message)
send_button.pack()

# Received message label
received_label = tk.Label(client, text="Received message:")
received_label.pack()

# Response label
response_label = tk.Label(client, text="")
response_label.pack()

# Start the receive thread
receive_thread = threading.Thread(target=receive_thread)
receive_thread.daemon = True
receive_thread.start()

# Start the client UI
client.mainloop()